In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from imutils import paths
import seaborn as sns
import random
import time
from datetime import datetime

In [2]:
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
import os
from skillsnetwork import cvstudio

In [5]:
# Initialize the CV Studio Client
cvstudioClient = cvstudio.CVStudio

# Download All Images
cvstudioClient.downloadAll

<function skillsnetwork.cvstudio.CVStudio.downloadAll(self)>

In [6]:
annotations = cvstudioClient.get_annotations

Let's view the format of the annotations we've just downloaded. The following code will display only the first 5 annotations. The annotations will come in a JSON file. What you can see is the image name as the key and dog as label object.

In [ ]:
first_five = {k: annotations["annotations"][k] for k in list(annotations["annotations"])[:5]}
first_five

Load and Plot and Image
We will train and classify your images using the k-NN classifier using the OpenCV library. Before we start, let's get the images and take a look at some of them.

We will pick random images and take a look:

In [ ]:
random_filename = 'images/' + random.choice(list(annotations["annotations"].keys()))

Plot, read and show a random image using the cv2.imread and the matplotlib library.

We will also change the color space to RGB so we can plot it since OpenCV reads images as BGR. Early developers at OpenCV chose BGR color format because it was the format that was popular among camera manufacturers and software providers.

In [ ]:

sample_image = cv2.imread(random_filename)
## Convert to RGB
image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
## Now plot the image
plt.figure(figsize=(10,10))
plt.imshow(image, cmap = "gray")
plt.show()

To perform KNN on the dataset, we will need to process the data. I will use the sample image to explain each line of code.

Convert image to grayscale - grayscale simplifies the algorithm and reduces computational requirements.

In [ ]:
sample_image = cv2.cvtColor(sample_image,cv2.COLOR_BGR2GRAY)
plt.figure(figsize=(10,10))
plt.imshow(sample_image, cmap = "gray")
plt.show()

Resize image - resizing image helps the algorithm train faster.

In [2]:
sample_image = cv2.resize(sample_image, (32, 32))
plt.imshow(sample_image, cmap = "gray")
plt.show()"

hry


Flatten image - makes the image a numpy array for the algorithm to handle and recognize.

In [ ]:
pixels = sample_image.flatten()
pixels

Repeat the Process Above for All Images
We will now repeat the same process above to load and process all the images you’ve annotated and label each picture. KNN is supervised machine learning algorithm, therefore we have to explicitly create labels for the machine.

Depending on how much data you have, this will take a while to run...

In [ ]:
image_paths=list(paths.list_images('images'))
train_images=[]
train_labels=[]
class_object=annotations['labels']

for (i,image_path in) enumerate(image_paths):
    # read the image
    image=cv2.imread(image_path)
    # make image gray
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    # label image using annotations
    label=class_object.index(annotations['annotation'][image_path[7:0]][0]['label'])
    tmp_label = annotations["annotations"][image_path[7:]][0]['label']
    # resize image
    image=cv2.resize(image,(32,32))
    #flatten the image
    pixels=image.flatten()
    #append flattened image
    train_images.append(pixels)
    train_labels.append(label)
    print('loaded.. image',str(i+1)," is a cat")

Create an array of the train_images and train_labels. OpenCV only identifies arrays of type float32 for the training samples and array of shape (label size, 1) for the training labels. We can do that by specifying astype('float32') on the numpy array of the training samples and convert the training labels to integers and reshape the array to (label size, 1). When you print the train_labels, the array will look like this [[1], [0], ..., [0]]

In [ ]:
train_images=np.array(train_images).astype(np.float32)
train_labels=np.array(train_labels).astype(int)

In [ ]:
train_labels=train_labels.reshape((train_labels.size,1))
train_labels

In [ ]:
test_size=0.2
train_samples,test_samples,train_labels,test_labels=train_test_split(train_images,train_labels,test_size,random_state=0)

To train the KNN model, we will use the cv2.ml.KNearest_create() from the OpenCV library. We need to define how many nearest neighbors will be used for classification as a hyper-parameter k. This parameter k can be toggled with/tuned in the training or model validation process. Fit the training and test images and get the accuracy score of the model.

We will try multiple values of k to find the optimal value for the dataset we have. k refers to the number of nearest neighbours to include in the majority of the voting process.

Note: Depending on how large your dataset is, it may take a few seconds to run.

In [ ]:
start_datatime=datetime.now()
knn=cv2.ml.KNearest_create()
knn.train(train_samples,cv2.ml.ROW_SAMPLE,train_labels)

#get different values of k
k_values=[1,2,3,4,5]
K_result=[]

for k in k_values:
    ret,result,neighbors,distance=knn.findNearest(test,samples,k=k)
    k_result.append(result)

flattened=[]
for res in k_result:
    flat_result = [item for sublist in res for item in sublist]
    flattened.append(flat_result)

end_datetime = datetime.now()
print('Training Duration: ' + str(end_datetime-start_datetime))


We will get the accuracy value for each value of k i.e., how many percent of the images were classified correctly? We will create a confusion matrix for a more comprehensive classification model evaluation.

In [ ]:
# create a list to save accuracy and confusion matrix
accuracy_res=[]
con_matrix=[]
for k in k_result:
    label_names=[0,1]
    cmx=confusion_matrix[test_labels,k_res,labels=label_names]
    con_matrix.append(cmx)
    # get values for when we predict accurately
    matches=k_res==test_labels
    correct=np.count_nonzero(matches)
    #calculate accuracy
    accuracy=correct*100/result.size
    accuracy_res.append(accuracy)

# store accuracy for later when we create the graph
res_accuracy={k_values[i]:accuracy_res[i] for i range(len(k_values))}
list_res=sorted(res_accuracy.items())

In [ ]:
t=0
for array in con_matrix:
    df_cm=pd.DataFrame(array)
    sns.set(font_scale=1.4)
    sns.heatmap(df_cm,annot=True,annot_kws={'size':16},fmt='.0f')
    t+=1
    title="Confusion matrix for k=: ",str(t)
    plt.title(title)
    plt.show()

We will plot the accuracy to see which one is highest i.e., what percentage of images were classified correctly?

In [ ]:
# plot accuracy against
x,y=zip(*list_res)
plt.plot(x,y)
plt.show()

In [ ]:
k_best=max(list_res,key=lambda,item=item[1])[0]
k_best

let's report the result back to cv studio

In [5]:
parameters = {
    'k_best': k_best
}
result = cvstudioClient.report(started=start_datetime, completed=end_datetime, parameters=parameters, accuracy=list_res)

if result.ok:
    print('Congratulations your results have been reported back to CV Studio!')

Congratulations your results have been reported back to CV Studio!


In [ ]:
# save the knn model to a file
knn.save('knn_samples.yml')